In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Sample_Inventory.csv')

In [5]:
df.head(10)

,Product,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,2024-01-09,...,2024-01-22,2024-01-23,2024-01-24,2024-01-25,2024-01-26,2024-01-27,2024-01-28,2024-01-29,2024-01-30,2024-01-31
0,product_1,130,126,123,121,116,115,108,104,102,...,60,53,46,39,35,30,25,19,18,16
1,product_2,252,250,246,244,240,240,235,230,225,...,189,185,179,173,172,167,164,158,156,151
2,product_3,146,142,137,132,127,125,123,120,117,...,73,69,65,59,53,47,45,41,41,40
3,product_4,267,267,263,260,256,254,253,252,246,...,215,212,211,208,206,202,198,193,190,187
4,product_5,256,250,247,245,240,237,234,229,223,...,184,180,177,175,175,170,164,163,160,160
5,product_6,215,215,215,213,212,207,203,199,194,...,151,148,144,139,137,131,128,122,119,113
6,product_7,209,198,189,178,165,154,146,133,121,...,0,0,0,0,0,0,0,0,0,0
7,product_8,151,145,137,130,124,116,111,109,103,...,41,34,26,23,16,9,5,1,0,0
8,product_9,256,255,255,252,247,244,241,238,236,...,181,177,171,171,166,161,159,157,157,153
9,product_10,124,122,118,114,114,113,107,101,99,...,67,61,58,55,55,53,47,41,39,38


In [21]:
def calculate_ewma(df, product, span=30):
    df_product = df.loc[df['Product'] == product].iloc[:,1:].T
    df_product.columns = ['Inventory']
    df_product['Inventory'] = df_product['Inventory'].astype(int)
    df_product['daily_usage'] = -df_product['Inventory'].diff()
    df_product['ewma'] = df_product['daily_usage'].ewm(span=span).mean()
    return df_product


In [75]:
products = df['Product'].unique()
result = []

for product in products:
    df_product = calculate_ewma(df, product)
    
    last_inventory = df_product.iloc[-1]['Inventory']
    last_ewma = df_product.iloc[-1]['ewma']
    
    # 재고 소진 예상일 계산
    expected_days = last_inventory / last_ewma
    
    # 5일 이내에 소진될 것으로 예상되면 EWMA 고려한 15일치 재고 보충 제안
    if expected_days <= 5:
        result.append([product,int(last_inventory),int(last_ewma*15)])


In [76]:
# List를 Data Frame으로 변환
df_result = pd.DataFrame(result)
df_result.columns = [['제품명','잔여 재고','보충 수량']]

In [77]:
df_result

,제품명,잔여 재고,보충 수량
0,product_1,16,59
1,product_7,0,68
2,product_8,0,69
3,product_14,9,100
4,product_22,9,45
5,product_23,0,75
6,product_25,0,97
7,product_27,0,70
8,product_28,9,52
9,product_33,13,43


In [63]:
df_result.to_csv('Restock_List.csv')